In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/csv/processed/record-1999.csv'). \
    drop('Unnamed: 0', axis=1)
data.head()

,branch,congressID,ideology_score,page,part,speaker,speech,year
0,NaN,106,0.391,1,1999-01-08,Mr. THURMOND.,I suggest the absence of a quorum. The PRESI...,1999
1,NaN,106,0.390,1,1999-01-08,Mr. LOTT.,"Mr. President, to inform all of our colleague...",1999
2,Senate,106,0.331,188,1999-01-14,Mr. BENNETT.,"Your Honor, excuse me, Mr. President, I need ...",1999
3,Senate,106,0.390,2,1999-01-22,Mr. LOTT.,"Mr. Chief Justice, I do send the first questi...",1999
4,Senate,106,-0.326,30,1999-01-22,Mr. ROCKEFELLER.,"Mr. President, I would like to join Senator F...",1999


In [3]:
remove_speakers = ['The PRESIDING OFFICER.',
                   'The SPEAKER pro tempore.']
data = data[~data.speaker.isin(remove_speakers)]

data['speech'] = data['speech'].str.replace('Mr\.', 'Mr')
data['speech'] = data['speech'].str.replace('Mrs\.', 'Mrs')
data['speech'] = data['speech'].str.replace('Ms\.', 'Ms')
data['speech'] = data['speech'].str.replace('\s+', ' ')
data['speech'] = data['speech'].str.replace('a\.m\. ', 'am ')
data['speech'] = data['speech'].str.replace('p\.m\. ', 'pm ')
data['speech'] = data['speech'].str.replace('H\.R\. ', 'HR ')

data.head()

,branch,congressID,ideology_score,page,part,speaker,speech,year
0,NaN,106,0.391,1,1999-01-08,Mr. THURMOND.,I suggest the absence of a quorum. The PRESID...,1999
1,NaN,106,0.390,1,1999-01-08,Mr. LOTT.,"Mr President, to inform all of our colleagues...",1999
2,Senate,106,0.331,188,1999-01-14,Mr. BENNETT.,"Your Honor, excuse me, Mr President, I need s...",1999
3,Senate,106,0.390,2,1999-01-22,Mr. LOTT.,"Mr Chief Justice, I do send the first questio...",1999
4,Senate,106,-0.326,30,1999-01-22,Mr. ROCKEFELLER.,"Mr President, I would like to join Senator FR...",1999


# Sentences

In [4]:
sentences = pd.concat([data['speech'].str.split('\. ', expand=True)])
df = pd.concat([data, sentences], axis=1). \
    drop('speech', axis=1)
    
df = pd.melt(df, id_vars=['branch','congressID','ideology_score','page','part', 'speaker', 'year'],
             value_name='sentence'). \
    sort_values(by=['year','part', 'page']). \
    reset_index(). \
    drop(['variable', 'index'], axis=1)
    
df['sent_length'] = df['sentence'].str.len()
df = df.query('sent_length > 8'). \
    drop('sent_length', axis=1)
    
df['sentence'] = df['sentence'].str.replace('VerDate.*', '')
    
df.head()

,branch,congressID,ideology_score,page,part,speaker,year,sentence
0,NaN,106,0.391,1,1999-01-08,Mr. THURMOND.,1999,I suggest the absence of a quorum
1,NaN,106,0.390,1,1999-01-08,Mr. LOTT.,1999,"Mr President, to inform all of our colleagues..."
2,NaN,106,0.391,1,1999-01-08,Mr. THURMOND.,1999,The PRESIDING OFFICER ENZI)
3,NaN,106,0.390,1,1999-01-08,Mr. LOTT.,1999,"And when a quorum is established, the Senate w..."
4,NaN,106,0.391,1,1999-01-08,Mr. THURMOND.,1999,The clerk will call the roll


In [5]:
pd.set_option('display.max_rows', 500)
pd.options.display.max_colwidth = 100

for i in df['sentence'][df['sentence'].str.contains('(.*[0-9]){4}')][0:50]:
    print(i,'\n')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


1503 states that obstruction of justice occurs when a person corruptly endeavors to influence the testimony of another person 

2217, the Federal Research Investment Act, and previous to that Senators DOMENICIand BINGAMAN, introduced S 

1305, the National Research Investment Act 

1305 and S 

2217 have been extremely successful in galvanizing members of the scientific and engineering community to pull together and work constructively towards a common ideal 

2217 passed without dissent in the Senate at the end of last session, and gained 36 co-sponsors—18 Democrats and 18 Republicans 

1305 and S 

2217 the trend, at least in the last two years, seems to have reversed and a universal spirit of cooperation for strong R&D funding is developing on all fronts 

In particular, for Fiscal Year 1999, an unprecedented 10% increase in civilian R&D funding was appropriated 

Based on a careful review and analysis of our past history, our bill authorizes an annual funding increase of 5.5%, star

In [7]:
df[df['sentence'].str.contains('VerDate')]

,branch,congressID,ideology_score,page,part,speaker,year,sentence
